### extrair termos e contagem para ser submetida ao software usagi.
 
 
 * pensar como tratar termos compartilhados entre os bancos , pois os dados depois voltam
 * como trabalhar tbm com ids confiaveis 
 
 Bancos: vacinas /SRAG
 
 variáveis que fiz em separado pois precisam ser harmonizadas em conjunto (precisam mesmo?) - Raça - Sexo

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from pyspark.sql import functions as F
import glob

In [2]:
vac  = spark.read.parquet("/dados10t/datalake/raw/vigivac/vacinacao/vacinacao17032023.parquet/")

In [5]:
vac.select("estabelecimento_valor").distinct().count()

36390

In [6]:
#vac.take(2)

In [7]:
vac.printSchema()

root
 |-- document_id: string (nullable = true)
 |-- id_paciente: string (nullable = true)
 |-- paciente_idade: double (nullable = true)
 |-- paciente_datanascimento: string (nullable = true)
 |-- paciente_enumsexobiologico: string (nullable = true)
 |-- paciente_racacor_codigo: string (nullable = true)
 |-- paciente_racacor_valor: string (nullable = true)
 |-- paciente_endereco_coibgemunicipio: string (nullable = true)
 |-- paciente_endereco_copais: string (nullable = true)
 |-- paciente_endereco_nmmunicipio: string (nullable = true)
 |-- paciente_endereco_nmpais: string (nullable = true)
 |-- paciente_endereco_uf: string (nullable = true)
 |-- paciente_endereco_cep: string (nullable = true)
 |-- paciente_nacionalidade_enumnacionalidade: string (nullable = true)
 |-- estabelecimento_valor: string (nullable = true)
 |-- estabelecimento_razaosocial: string (nullable = true)
 |-- estalecimento_nofantasia: string (nullable = true)
 |-- estabelecimento_municipio_codigo: string (nullable = 

In [8]:
vac.count()

550640907

In [9]:
vac.select("vacina_lote").distinct().count()

57262

In [10]:
vac.select("vacina_fabricante_nome").distinct().count()

25

In [11]:
vac.select("vacina_fabricante_referencia").distinct().count()

76

In [14]:
vac.select("vacina_nome").distinct().count()  

11

In [16]:
vac.groupBy("vacina_nome","vacina_fabricante_nome").count().orderBy(  F.col("count").desc()   ).show(150,False)

+------------------------------------------------------+-----------------------------------------+---------+
|vacina_nome                                           |vacina_fabricante_nome                   |count    |
+------------------------------------------------------+-----------------------------------------+---------+
|COVID-19 PFIZER - COMIRNATY                           |PFIZER                                   |211044986|
|COVID-19 ASTRAZENECA/FIOCRUZ - COVISHIELD             |ASTRAZENECA/FIOCRUZ                      |155270923|
|COVID-19 SINOVAC/BUTANTAN - CORONAVAC                 |SINOVAC/BUTANTAN                         |118922716|
|COVID-19 JANSSEN - Ad26.COV2.S                        |JANSSEN                                  |35493031 |
|COVID-19 PEDIÁTRICA - PFIZER COMIRNATY                |Pendente Identificação                   |9000896  |
|COVID-19 ASTRAZENECA - ChAdOx1-S                      |ASTRAZENECA                              |7980477  |
|COVID-19 PFIZER - 

In [4]:
vac_termo_op2 = vac.groupBy("vacina_nome").count().orderBy(  F.col("count").desc()   )
vac_termo_op2 = vac_termo_op2.withColumn("id_termo",F.monotonically_increasing_id())
vac_termo_op2.toPandas().to_csv("/dados10t/datalake/standard/omop/lista_v2_op2_termos_banco_vacinas_17032023.csv",sep=",",index=False)

In [22]:
vac_termos = vac.groupBy("vacina_nome","vacina_fabricante_nome").count().orderBy(  F.col("count").desc()   )

In [23]:
vac_termos = vac_termos.withColumn("id_termo",F.monotonically_increasing_id())

In [24]:
vac_termos.toPandas().to_csv("/dados10t/datalake/standard/omop/lista_v2_termos_banco_vacinas_17032023.csv",sep=",",index=False)

In [21]:
#vac.groupBy("vacina_fabricante_nome").count().orderBy(  F.col("count").desc()   ).show(50,False)

In [19]:
#vac.groupBy("vacina_fabricante_referencia").count().orderBy(  F.col("count").desc()   ).show(100,False)

In [ ]:
vac.groupBy("vacina_grupoatendimento_nome").count().orderBy(  F.col("count").desc()   ).show(50,False)

In [ ]:
vac_vacina_nome = vac.groupBy("vacina_nome").count().orderBy(  F.col("count").desc()   ).cache()

In [8]:
vac.groupBy("vacina_grupoatendimento_nome").count().orderBy(  F.col("count").desc()   ).show(50,False)

+-----------------------------------------------------------------------+---------+
|vacina_grupoatendimento_nome                                           |count    |
+-----------------------------------------------------------------------+---------+
|Pessoas de 18 a 64 anos                                                |272590074|
|Outros                                                                 |40820962 |
|Pessoas de 12 a 17 anos                                                |31377855 |
|Pessoas de 65 a 69 anos                                                |26408885 |
|Pessoas de 5 a 11 anos                                                 |24231689 |
|Pessoas de 70 a 74 anos                                                |19954052 |
|Hipertensão de difícil controle ou com complicações/lesão de órgão alvo|18059392 |
|Pessoas de 80 anos ou mais                                             |15519202 |
|Pessoas de 75 a 79 anos                                                |132

In [9]:
vac.groupBy("vacina_categoria_nome").count().orderBy(  F.col("count").desc()   ).show(50,False)

+-----------------------------------------------+---------+
|vacina_categoria_nome                          |count    |
+-----------------------------------------------+---------+
|Faixa Etária                                   |384975988|
|Trabalhadores de Saúde                         |52290225 |
|Comorbidades                                   |43783339 |
|null                                           |26076791 |
|Trabalhadores da Educação                      |12109652 |
|Outros                                         |8270560  |
|Trabalhadores Industriais                      |3669876  |
|Pessoas com Deficiência                        |2928481  |
|Trabalhadores de Transporte                    |2837072  |
|Povos e Comunidades Tradicionais               |2805130  |
|Pessoas de 60 anos ou mais institucionalizadas |2419964  |
|Gestantes                                      |2151492  |
|Povos Indígenas                                |1304717  |
|Forças de Segurança e Salvamento       

In [10]:
vac.groupBy("vacina_fabricante_nome").count().orderBy(  F.col("count").desc()   ).show(50,False)

+-----------------------------------------+---------+
|vacina_fabricante_nome                   |count    |
+-----------------------------------------+---------+
|PFIZER                                   |216355636|
|ASTRAZENECA/FIOCRUZ                      |155270923|
|SINOVAC/BUTANTAN                         |118922716|
|JANSSEN                                  |35493031 |
|Pendente Identificação                   |10500909 |
|ASTRAZENECA                              |7980477  |
|PFIZER - PEDIÁTRICA                      |3335295  |
|Pendente Identifica??o                   |1383328  |
|PFIZER - PEDIÁTRICA MENOR DE 5 ANOS      |1056941  |
|SINOVAC                                  |340136   |
|PFIZER MANUFACTURING BELGIUM NV - BELGICA|542      |
|FUNDACAO OSWALDO CRUZ                    |425      |
|JANSSEN PHARMACEUTICA NV                 |141      |
|COVID-19 JANSSEN - Ad26.COV2.S           |98       |
|ASTRAZENECA/OXFORD                       |94       |
|PFIZER/BIONTECH            

In [11]:
vac.groupBy("vacina_descricao_dose").count().orderBy(  F.col("count").desc()   ).show(50,False)

+------------------------------+---------+
|vacina_descricao_dose         |count    |
+------------------------------+---------+
|1ª Dose                       |193321480|
|2ª Dose                       |173274355|
|Reforço                       |122582884|
|2º Reforço                    |45137781 |
|Dose Adicional                |5381533  |
|Dose                          |4769177  |
|3ª Dose                       |2002624  |
|4ª Dose                       |1325244  |
|3º Reforço                    |1012476  |
|1º Reforço                    |574309   |
|Única                         |367416   |
|2ª Dose Revacinação           |211137   |
|4ª Dose Revacinação           |190459   |
|1ª Dose Revacinação           |165955   |
|3ª Dose Revacinação           |120493   |
|4º Reforço                    |115555   |
|Revacinação                   |68019    |
|5ª Dose                       |6887     |
|null                          |6012     |
|5ª Dose Revacinação           |5199     |
|Dose Inici

In [12]:
vac.groupBy("vacina_nome").count().orderBy(  F.col("count").desc()   ).show(50,False)

+------------------------------------------------------+---------+
|vacina_nome                                           |count    |
+------------------------------------------------------+---------+
|COVID-19 PFIZER - COMIRNATY                           |211045537|
|COVID-19 ASTRAZENECA/FIOCRUZ - COVISHIELD             |155271459|
|COVID-19 SINOVAC/BUTANTAN - CORONAVAC                 |118922823|
|COVID-19 JANSSEN - Ad26.COV2.S                        |35493277 |
|COVID-19 PEDIÁTRICA - PFIZER COMIRNATY                |12346210 |
|COVID-19 ASTRAZENECA - ChAdOx1-S                      |7980477  |
|COVID-19 PFIZER - COMIRNATY BIVALENTE                 |5310650  |
|COVID-19 SINOVAC - CORONAVAC                          |1842183  |
|COVID-19 PFIZER - COMIRNATY PEDIÁTRICA                |1371324  |
|COVID-19 PFIZER - COMIRNATY PEDIÁTRICA MENOR DE 5 ANOS|1056941  |
|INF3                                                  |26       |
+------------------------------------------------------+------

In [21]:
#vac_termos = vac.select("vacina_nome","vacina_descricao_dose","vacina_fabricante_nome","vacina_categoria_nome")

In [29]:
vac_vacina_nome = vac.groupBy("vacina_nome").count().orderBy(  F.col("count").desc()   ).cache()
vac_vacina_descricao_dose = vac.groupBy("vacina_descricao_dose").count().orderBy(  F.col("count").desc()   ).cache()
vac_vacina_fabricante_nome = vac.groupBy("vacina_fabricante_nome").count().orderBy(  F.col("count").desc()   ).cache()
vac_vacina_categoria_nome = vac.groupBy("vacina_categoria_nome").count().orderBy(  F.col("count").desc()   ).cache()

23/04/06 17:14:19 WARN CacheManager: Asked to cache already cached data.
23/04/06 17:14:19 WARN CacheManager: Asked to cache already cached data.
23/04/06 17:14:19 WARN CacheManager: Asked to cache already cached data.
23/04/06 17:14:19 WARN CacheManager: Asked to cache already cached data.


In [30]:
vac_vacina_nome = vac_vacina_nome.withColumn("variavel",F.lit("vacina_nome"))
vac_vacina_nome = vac_vacina_nome.withColumnRenamed("count","n_registro")
vac_vacina_nome = vac_vacina_nome.withColumnRenamed("vacina_nome","termo")


vac_vacina_descricao_dose = vac_vacina_descricao_dose.withColumn("variavel",F.lit("vacina_descricao_dose"))
vac_vacina_descricao_dose = vac_vacina_descricao_dose.withColumnRenamed("count","n_registro")
vac_vacina_descricao_dose = vac_vacina_descricao_dose.withColumnRenamed("vacina_descricao_dose","termo")


vac_vacina_fabricante_nome = vac_vacina_fabricante_nome.withColumn("variavel",F.lit("vacina_fabricante_nome"))
vac_vacina_fabricante_nome = vac_vacina_fabricante_nome.withColumnRenamed("count","n_registro")
vac_vacina_fabricante_nome = vac_vacina_fabricante_nome.withColumnRenamed("vacina_fabricante_nome","termo")


vac_vacina_categoria_nome = vac_vacina_categoria_nome.withColumn("variavel",F.lit("vacina_categoria_nome"))
vac_vacina_categoria_nome = vac_vacina_categoria_nome.withColumnRenamed("count","n_registro")
vac_vacina_categoria_nome = vac_vacina_categoria_nome.withColumnRenamed("vacina_categoria_nome","termo")

In [33]:
vac_vacina_categoria_nome.show(3,False)

+----------------------+----------+---------------------+
|termo                 |n_registro|variavel             |
+----------------------+----------+---------------------+
|Faixa Etária          |384975988 |vacina_categoria_nome|
|Trabalhadores de Saúde|52290225  |vacina_categoria_nome|
|Comorbidades          |43783339  |vacina_categoria_nome|
+----------------------+----------+---------------------+
only showing top 3 rows



In [35]:
vac_termos = vac_vacina_nome.unionByName(vac_vacina_descricao_dose)
vac_termos = vac_termos.unionByName(vac_vacina_fabricante_nome)
vac_termos = vac_termos.unionByName(vac_vacina_categoria_nome).cache()

In [36]:
vac_termos.count()

96

In [41]:
vac_termos = vac_termos.withColumn("id_termo",F.monotonically_increasing_id())

In [43]:
vac_termos.toPandas().to_csv("/dados10t/datalake/standard/omop/lista_v1_termos_banco_vacinas_17032023.csv",sep=",",index=False)

In [38]:
vac_termos.show(30,False)

+------------------------------------------------------+----------+---------------------+
|termo                                                 |n_registro|variavel             |
+------------------------------------------------------+----------+---------------------+
|COVID-19 PFIZER - COMIRNATY                           |211045537 |vacina_nome          |
|COVID-19 ASTRAZENECA/FIOCRUZ - COVISHIELD             |155271459 |vacina_nome          |
|COVID-19 SINOVAC/BUTANTAN - CORONAVAC                 |118922823 |vacina_nome          |
|COVID-19 JANSSEN - Ad26.COV2.S                        |35493277  |vacina_nome          |
|COVID-19 PEDIÁTRICA - PFIZER COMIRNATY                |12346210  |vacina_nome          |
|COVID-19 ASTRAZENECA - ChAdOx1-S                      |7980477   |vacina_nome          |
|COVID-19 PFIZER - COMIRNATY BIVALENTE                 |5310650   |vacina_nome          |
|COVID-19 SINOVAC - CORONAVAC                          |1842183   |vacina_nome          |
|COVID-19 

In [2]:
vac.printSchema()

root
 |-- document_id: string (nullable = true)
 |-- id_paciente: string (nullable = true)
 |-- paciente_idade: double (nullable = true)
 |-- paciente_datanascimento: string (nullable = true)
 |-- paciente_enumsexobiologico: string (nullable = true)
 |-- paciente_racacor_codigo: string (nullable = true)
 |-- paciente_racacor_valor: string (nullable = true)
 |-- paciente_endereco_coibgemunicipio: string (nullable = true)
 |-- paciente_endereco_copais: string (nullable = true)
 |-- paciente_endereco_nmmunicipio: string (nullable = true)
 |-- paciente_endereco_nmpais: string (nullable = true)
 |-- paciente_endereco_uf: string (nullable = true)
 |-- paciente_endereco_cep: string (nullable = true)
 |-- paciente_nacionalidade_enumnacionalidade: string (nullable = true)
 |-- estabelecimento_valor: string (nullable = true)
 |-- estabelecimento_razaosocial: string (nullable = true)
 |-- estalecimento_nofantasia: string (nullable = true)
 |-- estabelecimento_municipio_codigo: string (nullable = 